**Transfer learning using the VGG16 model:**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
BATCH_SIZE = 32

#take samples instead of whole dataset
train_ds, validation_ds, test_ds = tfds.load(
    'cats_vs_dogs',
    split = ["train[:30%]", "train[30%:40%]", "train[40%:50%]"],  #30%,10%,10%
    as_supervised = True,
    batch_size=BATCH_SIZE
    )
train_ds.cardinality(), validation_ds.cardinality(), test_ds.cardinality()  #7k,2k,2k

**Plot few images:**

In [ ]:
plt.figure(figsize=(6,6))
for i, (img,lbl) in enumerate(train_ds.unbatch().take(6)):
  plt.subplot(2,3, i+1)
  plt.imshow(img)
  plt.title(int(lbl))
  plt.axis("off")

**Resize, augment, normalize:**

In [ ]:
IMG_SIZE = 150

#resize
resize = tf.keras.layers.Resizing(IMG_SIZE, IMG_SIZE)

#normalize - convert img from RGB to BGR, and zero-center each color channel
normalize = tf.keras.applications.vgg16.preprocess_input

train_augment = tf.keras.Sequential([
    resize,
    #augment
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1)
])

train_ds = train_ds.map(lambda x,y: (normalize(train_augment(x)),y))
validation_ds = validation_ds.map(lambda x,y: (normalize(resize(x)) ,y))
test_ds = test_ds.map(lambda x,y: (normalize(resize(x)) ,y))

**Model architecture** (using pre-trained VGG16 model):

In [ ]:
import keras.applications.vgg16 as vgg16
import keras.layers as layers

base_model = vgg16.VGG16(include_top=False, input_shape=(IMG_SIZE,IMG_SIZE,3), pooling="avg")
base_model.trainable = False  #freeze

model = tf.keras.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation="sigmoid")
])
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.SGD(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])
history = model.fit(train_ds, validation_data=validation_ds, epochs=15, batch_size=BATCH_SIZE)

**Plot accuracy and loss:**

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='lower left')
plt.show()

**Test and Evaluate:**

In [ ]:
loss, accuracy = model.evaluate(test_ds)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

In [ ]:
def predict_and_display_images(dataset, model, num_images=6):
  """Predicts categories and displays images with predictions and confidence."""

  plt.figure(figsize=(10, 10))
  for i, (img, label) in enumerate(dataset.unbatch().take(num_images)):
    # Add a batch dimension for prediction
    img_with_batch = tf.expand_dims(img, axis=0)
    prediction = model.predict(img_with_batch)

    plt.subplot(3, 2, i + 1)
    #normalized, won't show actual images
    display_img = img

    plt.imshow(display_img.numpy().astype("uint8"))

    predicted_class = "Dog" if prediction[0][0] > 0.5 else "Cat"
    confidence = prediction[0][0] if prediction[0][0] > 0.5 else 1 - prediction[0][0]

    plt.title(f"Predicted: {predicted_class}\nConfidence: {confidence:.2f}\nTrue: {'Dog' if label.numpy() else 'Cat'}")
    plt.axis("off")

  plt.tight_layout()
  plt.show()

predict_and_display_images(test_ds, model)